<a href="https://colab.research.google.com/github/mgerlach/advent_of_code/blob/main/2024/aoc2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Global generic utils

In [112]:
def vec_add(v1, v2):
  return tuple(sum(i) for i in zip(v1, v2))

def vec_sub(v1, v2):
  return tuple(i[0] - i[1] for i in zip(v1, v2))

dirs45 = [(dy, dx) for dy in [-1, 0, 1] for dx in [-1, 0, 1] if dy != 0 or dx != 0]

dir_up = (-1, 0)
dir_right = (0, 1)
dir_down = (1, 0)
dir_left = (0, -1)

dirs90 = [dir_up, dir_right, dir_down, dir_left]
turn_right = {dir_up: dir_right, dir_right: dir_down, dir_down: dir_left, dir_left: dir_up}
turn_left = {dir_up: dir_left, dir_left: dir_down, dir_down: dir_right, dir_right: dir_up}

def in_range(p, dimensions):
  y, x = p
  height, width = dimensions
  return y >= 0 and y < height and x >= 0 and x < width

def get_cell(p, grid):
  y, x = p
  return grid[y][x]


Day 01, input

In [ ]:
input01 = [[int(i) for i in line.split()] for line in open('drive/MyDrive/AoC/2024/input01.txt')]

Day 01, part 1, sum of absolute diff of sorted list elements

In [ ]:
sum(abs(l - r) for (l, r) in zip(sorted(l for (l, _) in input01), sorted(r for (_, r) in input01)))

1530215

Day 01, part 2, sum lhs elements multiplied by frequency in rhs list

In [ ]:
from collections import Counter
r_counts = Counter(r for (_, r) in input01)
sum(l * r_counts[l] for (l, _) in input01)

26800609

Day 02, input

In [ ]:
input02 = [[int(i) for i in line.split()] for line in open('drive/MyDrive/AoC/2024/input02.txt')]

Day 02, utils

In [ ]:
def same_sgn(deltas_line):
  return all(deltas_line[i] * deltas_line[i+1] > 0 for i in range(len(deltas_line)-1))

def delta_max(deltas_line, m):
  return all(abs(d) <= m for d in deltas_line)

Day 02, part 1, determine (all increasing or all decreasing) and deltas < 4

In [ ]:
deltas = [[line[n+1] - line[n] for n in range(len(line)-1)] for line in input02]

sum(1 for deltas_line in deltas if same_sgn(deltas_line) and delta_max(deltas_line, 3))

359

Day 02, part 2, allow removal of any single element

In [ ]:
def check_line(line):
  deltas = [line[i+1] - line[i] for i in range(len(line)-1)]
  return same_sgn(deltas) and delta_max(deltas, 3)

def remove1(line):
  return [line[:i] + line[i+1:] for i in range(len(line))]

# part 1 regression
# print(sum(1 for line in input02 if check_line(line)))

# part 2
sum(1 for line in input02 if check_line(line) or any(check_line(r) for r in remove1(line)))

418

Day 03, input

In [ ]:
input03 = "".join(line for line in open('drive/MyDrive/AoC/2024/input03.txt'))

Day 03, utils

In [ ]:
import re

def find_mul_and_eval(instructions):
  # regex mul\\((\\d+),(\\d+)\\)
  matches = re.findall("mul\((\d+),(\d+)\)", instructions)
  return sum(int(x) * int(y) for (x, y) in matches)

Day 03, part 1, find mul(x,y) sequences, multiply and add

In [ ]:
# input03 = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
find_mul_and_eval(input03)

162813399

Day 03, part 2, evaluate do() and don't() sequences

In [ ]:
import re
from functools import reduce

def next_state(state, split):
  s, is_active = state
  match split:
    case "do()":
      return (s, True)
    case "don't()":
      return (s, False)
    case _:
      return (s + (find_mul_and_eval(split) if is_active else 0), is_active)

# input03 = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
# split on (do\\(\\)|don't\\(\\)) - capturing group leads to delimiters being included in result
s, is_active = reduce(next_state, re.split("(do\(\)|don't\(\))", input03), (0, True))
s

53783319

Day 04, input

In [ ]:
input04 = [line.strip() for line in open('drive/MyDrive/AoC/2024/input04.txt')]
rows = len(input04)
cols = len(input04[0])
dimensions = (rows, cols)

Day 04, part 1, find XMAS

In [ ]:
def check_char(p, char):
  y, x = p
  return in_range(p, dimensions) and input04[y][x] == char

def check(s, p, d):
  return s == "" or check_char(p, s[0]) and check(s[1:], vec_add(p, d), d)

def count_xmas(p):
  return sum(1 for d in dirs45 if check("XMAS", p, d))

sum(count_xmas((y, x)) for y in range(rows) for x in range(cols))

2567

Day 04, part 2, find
```
M.S
.A.
M.S
```



In [ ]:
# The pattern can only occur in 4 different variants, with 'A' at (0,0)
patterns = [
  (((-1, -1), 'M'), ((-1, 1), 'S'), ((1, -1), 'M'), ((1, 1), 'S')),
  (((-1, -1), 'M'), ((-1, 1), 'M'), ((1, -1), 'S'), ((1, 1), 'S')),
  (((-1, -1), 'S'), ((-1, 1), 'M'), ((1, -1), 'S'), ((1, 1), 'M')),
  (((-1, -1), 'S'), ((-1, 1), 'S'), ((1, -1), 'M'), ((1, 1), 'M'))
]

# variant without range check
def check_char_unsafe(p, char):
  return get_cell(p, input04) == char

def check_pattern(p, pattern):
  return all(check_char_unsafe(vec_add(p, d), char) for (d, char) in pattern)

# Search for A within (1, 1)...(rows-1, cols-1) and check match for all patterns
sum(1
    for p in [(y, x) for y in range(1, rows-1) for x in range(1, cols-1) if input04[y][x] == 'A']
    for pattern in patterns if check_pattern(p, pattern))

2029

Day 05, input

In [ ]:
input05 = [line.strip() for line in open('drive/MyDrive/AoC/2024/input05.txt')]
split_index = input05.index('')
rules = [tuple(int(n) for n in rule.split('|')) for rule in input05[:split_index]]
updates = [[int(n) for n in update.split(',')] for update in input05[split_index+1:]]

Day 05, utils


In [ ]:
def verify_pair(left, right):
  """if a rule exists, check order against it, otherwise pass"""
  existing_rules = [rule for rule in rules if left in rule and right in rule]
  return not existing_rules or any(rl == left and rr == right for rl, rr in existing_rules)

def verify_update(update):
  return all(verify_pair(update[i], update[j]) for i in range(len(update)-1) for j in range(i+1, len(update)))

Day 05, part 1, find correct updates, sum up middle their elements

In [ ]:
sum(update[int(len(update)/2)] for update in updates if verify_update(update))

7365

Day 05, part 2, fix updates which break the rules, sum up their middle elements

In [ ]:
def fix_update(update_rest, fixed):
  if not update_rest:
    return fixed
  if not fixed:
    return fix_update(update_rest[1:], [update_rest[0]])
  # find correct place for update_rest[0] in fixed
  for i in range(len(fixed)):
    if verify_pair(update_rest[0], fixed[i]):
      return fix_update(update_rest[1:], fixed[:i] + [update_rest[0]] + fixed[i:])
  return fix_update(update_rest[1:], fixed + [update_rest[0]])

sum(fix_update(update, [])[int(len(update)/2)] for update in updates if not verify_update(update))

5770

Day 06, input

In [40]:
input06 = [line.strip() for line in open('drive/MyDrive/AoC/2024/input06.txt')]
height = len(input06)
width = len(input06[0])
dimensions = (height, width)
start = [(y, x) for y in range(height) for x in range(width) if get_cell((y, x), input06) == '^'][0]

Day 06, part 1, visisted cells

In [46]:
def get_visited(grid) -> tuple[list[tuple[tuple[int, int], tuple[int, int]]], bool]:
  dir = dir_up
  p = start
  visited = {(p, dir)}
  while(in_range(p, dimensions)):
    next = vec_add(p, dir)
    if in_range(next, dimensions):
      if get_cell(next, grid) == '#':
        dir = turn_right[dir]
      elif (next, dir) in visited:
        return (visited, True)
      else:
        p = next
        visited.add((p, dir))
    else:
      return (visited, False)

visited, _ = get_visited(input06)
len({p for (p, dir) in visited})

4711

Day 06, part 2, find loops

In [59]:
mutable = [[c for c in row] for row in input06]
loops = 0
for y in range(height):
  # print(y, loops)
  for x in range(width):
    if get_cell((y, x), input06) == '.':
      mutable[y][x] = '#'
      _, loop = get_visited(mutable)
      loops += loop
      mutable[y][x] = '.'
loops

1562

Day 07, input

In [72]:
# [(desired_result, [operand1, operand2, ...])]
input07 = [
    (int(result.strip()), [int(operand.strip()) for operand in operands.split()])
    for (result, operands) in [[s.strip() for s in line.split(':')] for line in open('drive/MyDrive/AoC/2024/input07.txt')]]

Day 07, part 1, find valid equations with '*' and '+'

In [82]:
def check_equation(desired_result, current_result, remaining_operands):
  if not remaining_operands:
    return desired_result == current_result
  if current_result > desired_result:
    return False
  return check_equation(desired_result, current_result + remaining_operands[0], remaining_operands[1:]) or \
         check_equation(desired_result, current_result * remaining_operands[0], remaining_operands[1:])

sum(desired_result for (desired_result, operands) in input07 if check_equation(desired_result, operands[0], operands[1:]))

4555081946288

Day 07, part 2, find valid equations with '*', '+', '||' (concat)

In [83]:
from math import log10
def check_equation2(desired_result, current_result, remaining_operands):
  if not remaining_operands:
    return desired_result == current_result
  if current_result > desired_result:
    return False
  return check_equation2(desired_result, current_result + remaining_operands[0], remaining_operands[1:]) or \
         check_equation2(desired_result, current_result * remaining_operands[0], remaining_operands[1:]) or \
         check_equation2(desired_result, current_result * 10 ** (int(log10(remaining_operands[0])) + 1) + remaining_operands[0], remaining_operands[1:])
         # or with strings: check_equation2(desired_result, int(f'{current_result}{remaining_operands[0]}'), remaining_operands[1:])

sum(desired_result for (desired_result, operands) in input07 if check_equation2(desired_result, operands[0], operands[1:]))

227921760109726

Day 08, input

In [139]:
# get width, height
input08 = [line.strip() for line in open('drive/MyDrive/AoC/2024/input08.txt')]
height = len(input08)
width = len(input08[0])
dimensions = (height, width)
antennas = {}
for (name, pos) in [(input08[y][x], (y, x)) for y in range(height) for x in range(width) if input08[y][x] != '.']:
  if name in antennas:
    antennas[name].append(pos)
  else:
    antennas[name] = [pos]


Day 08, part 1, find antinodes (unique positions)

In [140]:
def antinodes(positions):
  return [p for p in [vec_sub(p1, vec_sub(p2, p1)) for p1 in positions for p2 in positions if p1 != p2] if in_range(p, dimensions)]

len({p for a in antennas.values() for p in antinodes(a)})

354

Day 08, part 2, find more antinodes (unique positions)


In [141]:
def antinodes2(positions):
  def antinodes_for_pair(p1, p2):
    anodes = []
    dir = vec_sub(p2, p1)  # step
    p = p2  # p1, p2 are antinodes themselves, but no need to add p1 as we also call this for the swapped pair
    while(in_range(p, dimensions)):
      anodes.append(p)
      p = vec_add(p, dir)
    return anodes
  return {a for p1 in positions for p2 in positions if p1 != p2 for a in antinodes_for_pair(p1, p2)}

len({a for p in antennas.values() for a in antinodes2(p)})

1263